In [20]:
import pandas as pd
import numpy as np
import re
from pandas import DataFrame
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

pd.set_option('display.width', 300)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 100)
field_names = ["msisdn", "dialed", "start_time", "duration", "fraud", "prs"]
types = [str,str,str,int,int,int]
dtypes = dict(zip(field_names,types))
data = pd.read_csv(r'D:/ex/mtt_in/03/cdr.csv',',',names=field_names, dtype = dtypes, low_memory=False)
data._get_numeric_data()
#data['start_time'] = data.start_time.str.replace(r'\D', '')

data.head(2)

,msisdn,dialed,start_time,duration,fraud,prs
0,78462428145,79067199591,2016-03-20 00:02:52,22,0,0
1,74845150825,79998042528,2016-03-20 00:03:57,65,0,0


In [21]:
number = data.dialed.astype(str)
def get_area_code(num:str, localcode = False):
    
    return_code = ['','']
    #if len(num) == 11 and re.match(r'^7\d{10}',num):
    
    if len(num) == 11 and re.match(r'^7\d+',num):
        return_code = [num[:1]]+[num[1:4]]
        
    elif len(num) == 11 and num[0] != '7':
        return_code = [num[:3],'']
        
    elif len(num) > 11 and re.match(r'^1|2|3|4|5|6|7|8|9\d+',num):
        return_code = [num[:3]] + [num[3:5]]
    
    if not localcode:
        return return_code[0]
    else:
        return return_code[1]

for d in number:
    if re.match(r'[7][0-9]{10}', d) and len(d) == 11 and d.startswith('7'):
        country_code =  d[:1] 
        local_code = d[1:4]
    elif d[0] != '7' and len(d) == 11:
        country_code =  d[:3] 
    elif re.match(r'[3][0-9]{10}', d) and len(d) > 11:
        country_code =  d[:3] 
        local_code = d[3:5]
        number = d[-7:]

In [22]:
data['order'] = data.groupby(level=0)['start_time'].min().apply(lambda x: x[:10])
data['period_d'] = data.groupby(level=0)['start_time'].min().apply(lambda x: x[8:10])
data['period_h'] = data.groupby(level=0)['start_time'].min().apply(lambda x: x[11:-6])
data['count'] = data['start_time'].apply(lambda x: x[2:-16]).astype(int)       
data['country_code'] = data['dialed'].apply(lambda x:get_area_code(str(x)))
data['local_code'] = data['dialed'].apply(lambda x:get_area_code(str(x),localcode=True))

data.set_index('msisdn', inplace=True)
data['JoinMonth'] = data.groupby(level=0)['start_time'].min().apply(lambda x: x[0:7])
#data['JoinMonth'] = data.JoinMonth.str.replace(r'\D', '')

data.reset_index(inplace=True)
data.insert(len(data.columns), "TotalCalls", 0, allow_duplicates=False)

data['int'] = data['country_code'].apply(lambda x: 0 if x == '7' else 1)
data['alarm'] = data['duration'].apply(lambda x: 1 if x > 575 else 0)


group_counts = data.groupby(['JoinMonth', 'period_h','period_d','msisdn','int','alarm','prs','fraud','country_code'])
cohorts = group_counts.agg({'msisdn': pd.Series.nunique,
                            'dialed': pd.Series.nunique,
                            'TotalCalls': pd.Series.count,
                            'country_code': pd.Series.count,
                            'count':np.sum,
                            'duration': np.sum})

cohorts['TotalCalls'].unstack(1).replace(np.NaN,0).astype(int).to_csv('D:/ex/mtt_out/cdr_cohorts_hour_0001.csv')
cohorts['TotalCalls'].unstack(1).replace(np.NaN,0).head(100)

group_duration = data.groupby(by=['JoinMonth', 'period_h','period_d','msisdn','int','alarm','prs','fraud','country_code'])['duration'].sum().unstack(1).replace(np.NaN,0).astype(int).to_csv('D:/ex/mtt_out/cdr_cohorts_hour_0002.csv')

#att = cohorts['TotalCalls'].apply(lambda x: 1 if x > 5 else 0).to_csv('D:/ex/mtt_out/cdr_cohorts_hour_0003.csv')

#att = cohorts.groupby(by=['msisdn'])['TotalCalls'].apply(lambda x: 1 if x > 5 else 0).to_csv('D:/ex/mtt_out/cdr_cohorts_hour_0003.csv')



In [23]:
df1 = pd.read_csv(r'D:/ex/mtt_out/cdr_cohorts_hour_0001.csv',',')#, index_col = None)
df2 = pd.read_csv(r'D:/ex/mtt_out/cdr_cohorts_hour_0002.csv',',')#, index_col = None)
df1['country_code'] = df1['country_code'].replace(np.NaN,7).astype(int)
df2['country_code'] = df1['country_code'].replace(np.NaN,7).astype(int)
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)
df1['w'] = df1['int'] + df1['alarm'] + df1['prs'] + df1['fraud'] + df1['country_code']
df1.head(10)

,index,JoinMonth,period_d,msisdn,int,alarm,prs,fraud,country_code,00,01,02,03,04,05,...,10,11,12,13,14,15,16,17,18,19,20,21,22,23,w
0,0,2016-03,20,73012213297,0,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,7
1,1,2016-03,20,73012443905,0,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2,0,0,7
2,2,2016-03,20,73012445182,0,0,0,0,7,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,7
3,3,2016-03,20,73012454377,0,0,0,0,7,0,0,0,0,0,0,...,1,1,1,0,0,0,0,1,0,0,0,0,0,0,7
4,4,2016-03,20,73412500907,0,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,5,0,0,0,7
5,5,2016-03,20,73412500907,0,1,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,8
6,6,2016-03,20,73412904239,0,0,0,0,7,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,7
7,7,2016-03,20,73412911766,0,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1,0,0,0,0,7
8,8,2016-03,20,73412917966,0,0,0,0,7,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0,0,1,0,0,7
9,9,2016-03,20,73412956210,0,0,0,0,7,0,0,0,0,0,0,...,1,0,1,2,1,2,0,0,0,1,0,0,0,0,7


In [24]:
profile = pd.merge(df1,df2, on='index', how='outer')
profile.to_csv('D:/ex/mtt_out/cdr_profile.csv')

In [ ]:
#profile = pd.concat([df1,df2], axis=1)
#profile.head()

In [ ]:
profile.T.drop_duplicates().T.to_csv('D:/ex/mtt_out/cdr_profile.csv')

In [ ]:
profile.head()